In [1]:
import pandas as pd
import numpy
from __future__ import division

In [2]:
df3_WS2T = pd.read_csv("/Users/asteriag/Desktop/DATASCIENCEPROJECT_WORKFLOW/TimeStamp_Engineering/engineering_AF_segment3_workflowS2_has_duration.csv",engine='python')

In [3]:
df3_WS2T.head(n=1)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,time_used_str,time_used_second
0,1076747|438001|677047,Send for Assessment,Send for Assessment,11/1/2019 14:46:32,0 days 2 hours 18 minutes 51 seconds,8331.0


In [4]:
df3_WS2T['WRKFLW_TSK_MAP_GROUPED'].nunique()

55

In [5]:
df3_WS2T['WRKFLW_TSK_MAP_DETAILED'].nunique()

67

In [6]:
wd_count = df3_WS2T.groupby('WRKFLW_TSK_MAP_DETAILED').WRKFLW_TSK_MAP_DETAILED.count()
wd_dict = dict()
for i, v in wd_count.items():
    wd_dict[i] = v
sorted_wd_dict = sorted(wd_dict.items(), key = lambda x: x[1], reverse = True)
print(sorted_wd_dict)

[('Send for Assessment', 638), ('Assessment Decision - Qualified', 312), ('Assessment Outcome - Successful', 242), ('Assessment Outcome - Higher Preference Successful', 146), ('Assessment Decision - No Decision', 125), ('Assessment Outcome - Not Successful', 120), ('Acacdemic - Assessment Decision - Qualified', 116), ('Put Assessment On-Hold', 100), ('Assessment Decision - Not Qualified', 89), ('Release Assessment Hold', 88), ('End My Assessment', 72), ('OC - Created - Evidence of course completion', 39), ('AR - Satisfied - Academic results for prior tertiary studies', 37), ('AR - Satisfied - English language proficiency', 34), ('Back to Triage', 26), ('Assessment Outcome - Ineligible / Withdrawn', 25), ('Acacdemic - Assessment Decision - Not Qualified', 23), ('AR - Created - Assessment information (Coursework) (1)', 17), ('OC - Satisfied - Evidence of course completion', 16), ('Put Triage On-Hold', 13), ('Release Triage Hold', 12), ('AR - Created - Further information (1)', 10), ('AR 

In [7]:
list = df3_WS2T.WRKFLW_TSK_MAP_DETAILED.unique()
avg_time_dict = dict()
for key in list :
    df_slides = df3_WS2T[df3_WS2T['WRKFLW_TSK_MAP_DETAILED'] == key]
    sum = df_slides['time_used_second'].sum()
    c = len(df_slides)
    avg = sum//c
    avg_time_dict[key] = avg

sorted_avg_time_dict = sorted(avg_time_dict.items(),key = lambda x: x[1], reverse = True)
print(sorted_avg_time_dict)

[('Back to Perform Assessment', 22305657.0), ('AR - Satisfied - Graduate Access Melbourne', 3974422.0), ('AR - Created - Specialisation (Master of Engineering)', 2428683.0), ('OC - Created - Evidence of course completion', 2203604.0), ('AR - Requested - Further information (1)', 2109580.0), ('Put Triage On-Hold', 2015347.0), ('End My Assessment', 1524571.0), ('AR - Satisfied - Credit for previous study', 1448148.0), ('AR - Created - Assessment information (Coursework) (2)', 1430959.0), ('AR - Satisfied - University of Melbourne Statement of Results / transcript', 1321161.0), ('AR - Created - Further information (2)', 1105686.0), ('Send for Assessment', 1099679.0), ('AR - Satisfied - Official syllabus or subject descriptions (in English)', 1059978.0), ('AR - Satisfied - Current enrolment at the University of Melbourne', 1036606.0), ('Assessment Outcome - Higher Preference Successful', 965724.0), ('Put Assessment On-Hold', 921397.0), ('Release Assessment Hold', 882701.0), ('AR - Satisfie

In [8]:
numOfWorkflowProcesses = df3_WS2T.groupby('CASE_ID').CASE_ID.count()

count_dic=dict()
for i,v in numOfWorkflowProcesses.items():
    if count_dic.get(v) is None:
        count_dic[v]=[]
        count_dic[v].append(i)
    else:
        count_dic[v].append(i)
#count_dic
count_dic = dict(sorted(count_dic.items()))

In [9]:
len_list=[]
for key in count_dic.keys():
    len_list.append(len(count_dic[key]))

In [10]:
def strfdelta(tdelta, fmt):
    d=dict()
    d["days"],rem1 = divmod(tdelta,60*60*24)
    d["hours"], rem2 = divmod(rem1, 3600)
    d["minutes"], d["seconds"] = divmod(rem2, 60)
    return fmt.format(**d)

In [11]:
list_avgtime_per_key =[]
list_sumtime_row = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_person = 0
    for id in id_list:
        df = df3_WS2T[df3_WS2T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_person+=tsum
    list_avgtime_per_key.append(int((list_sumtime_person/len(count_dic[key]))))
    list_sumtime_row.append(list_sumtime_person)
list_avgtime_per_key_str = []
for i in list_avgtime_per_key:
    list_avgtime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [12]:
list_mediantime_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_per_person = []
    for id in id_list:
        df = df3_WS2T[df3_WS2T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_per_person.append(tsum)
    list_mediantime_per_key.append(numpy.median(list_sumtime_per_person))

list_mediantime_per_key_str = []
for i in list_mediantime_per_key:
    list_mediantime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [13]:
list_percent_reach_end = []
for key in count_dic.keys():
    id_list = count_dic[key]
    count_df_end_step = 0
    count_df_id = 0
    for id in id_list:
        df_id = df3_WS2T[df3_WS2T['CASE_ID']== id]
        df_end_step = df_id[df_id['WRKFLW_TSK_MAP_GROUPED'].isin(['Assessment Outcome - Successful','Assessment Outcome - Not Successful']) ]
        count_df_id += df_id['CASE_ID'].nunique()
        count_df_end_step += df_end_step['CASE_ID'].nunique()
    list_percent_reach_end.append(str(round((count_df_end_step/count_df_id)*100,2))+ "%")

In [14]:
list_step_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = set()
    for id in id_list:
        df = df3_WS2T[df3_WS2T['CASE_ID']== id]
        for index, row in df.iterrows():
            list_step.add(row['WRKFLW_TSK_MAP_GROUPED'])
    list_step_per_key.append(list_step)

In [15]:
from collections import Counter
import collections, functools, operator
list_step_sum_time_per_key = []
list_step_count_per_key = []
list_step_ave_time_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = []
    step_time_list = []
    for id in id_list:
        df = df3_WS2T[df3_WS2T['CASE_ID']== id]
        step_time = dict(zip(df.WRKFLW_TSK_MAP_DETAILED, df.time_used_second))
        step_time_list.append(step_time)
        for index, row in df.iterrows():
            list_step.append(row['WRKFLW_TSK_MAP_DETAILED'])

    step_sum_time = dict(functools.reduce(operator.add,
         map(collections.Counter, step_time_list)))

    dict_list_step=dict(Counter(list_step))

    step_ave_time = {key: step_sum_time[key] // dict_list_step.get(key, 0)
                        for key in step_sum_time.keys()}

    sort_step_sum_time = sorted(step_sum_time.items(),key = lambda x: x[1], reverse = True)
    list_step_sum_time_per_key.append(sort_step_sum_time)

    sort_dict_list_step = sorted(dict_list_step.items(),key = lambda x: x[1], reverse = True)
    list_step_count_per_key.append(sort_dict_list_step[:6])

    sort_step_ave_time = sorted(step_ave_time.items(),key = lambda x: x[1], reverse = True)
    list_step_ave_time_per_key.append(sort_step_ave_time)

In [16]:
s3_t2=pd.DataFrame({'Number of Workflow processes in this Workflow Segment':count_dic.keys(),'Number of cases':len_list, 'average time for workflow segment':list_avgtime_per_key_str,'median time for workflow segment':list_mediantime_per_key_str,'percentage of cases reaching to end of workflow segment':list_percent_reach_end})
s3_t2['Num of Occurrences per step(descending)'] = list_step_count_per_key
s3_t2['Average time spends per step(descending)'] = list_step_ave_time_per_key
s3_t2['average time for workflow segment(seconds)'] = list_avgtime_per_key
s3_t2['sum time for workflow segment(seconds)'] = list_sumtime_row
s3_t2.head(n=10)


,Number of Workflow processes in this Workflow Segment,Number of cases,average time for workflow segment,median time for workflow segment,percentage of cases reaching to end of workflow segment,Num of Occurrences per step(descending),Average time spends per step(descending),average time for workflow segment(seconds),sum time for workflow segment(seconds)
0,1,46,0 days 0 hours 0 minutes 0 seconds,0.0 days 0.0 hours 0.0 minutes 0.0 seconds,0.0%,"[(Send for Assessment, 46)]",[],0,0.0
1,2,133,16 days 20 hours 26 minutes 30 seconds,9.0 days 22.0 hours 5.0 minutes 1.0 seconds,32.33%,"[(Send for Assessment, 133), (Assessment Outco...","[(Send for Assessment, 1455990.0)]",1455990,193646731.0
2,3,141,27 days 6 hours 49 minutes 34 seconds,12.0 days 5.0 hours 57.0 minutes 23.0 seconds,68.79%,"[(Send for Assessment, 142), (Assessment Outco...",[(OC - Created - Evidence of course completion...,2357374,332389795.0
3,4,116,48 days 8 hours 9 minutes 46 seconds,24.0 days 23.0 hours 55.0 minutes 7.0 seconds,68.97%,"[(Send for Assessment, 116), (Assessment Decis...",[(OC - Created - Evidence of course completion...,4176586,484484005.0
4,5,67,39 days 13 hours 22 minutes 35 seconds,28.0 days 2.0 hours 7.0 minutes 24.0 seconds,74.63%,"[(Send for Assessment, 67), (Assessment Decisi...","[(AR - Satisfied - Graduate Access Melbourne, ...",3417755,228989628.0
5,6,34,38 days 14 hours 19 minutes 54 seconds,35.0 days 3.0 hours 9.0 minutes 20.0 seconds,79.41%,"[(Assessment Decision - Qualified, 40), (Send ...","[(Put Triage On-Hold, 4166412.0), (AR - Not Sa...",3334794,113383029.0
6,7,22,61 days 4 hours 58 minutes 31 seconds,22.0 days 2.0 hours 30.0 minutes 23.0 seconds,68.18%,"[(Assessment Decision - Qualified, 25), (Send ...","[(Back to Perform Assessment, 22305657.0), (AR...",5288311,116342852.0
7,8,18,33 days 12 hours 8 minutes 45 seconds,23.0 days 15.0 hours 2.0 minutes 9.0 seconds,83.33%,"[(Assessment Decision - Qualified, 21), (Send ...","[(AR - Created - Further information (2), 3626...",2894925,52108666.0
8,9,17,38 days 18 hours 32 minutes 37 seconds,34.0 days 20.0 hours 2.0 minutes 21.0 seconds,94.12%,"[(Assessment Decision - Qualified, 30), (Send ...","[(AR - Satisfied - Further information (1), 17...",3349957,56949275.0
9,10,7,40 days 3 hours 29 minutes 45 seconds,19.0 days 0.0 hours 35.0 minutes 4.0 seconds,85.71%,"[(Assessment Decision - No Decision, 14), (Ass...","[(Put Triage On-Hold, 15573013.0), (AR - Satis...",3468585,24280100.0


In [17]:
s3_t2.to_csv('engineering_AF_segment3_workflowS2_table.csv', index=False)



